In [35]:
!pip install transformers[torch]
!pip install PyPDF2==2.9.0
!python3 -m nltk.downloader averaged_perceptron_tagger
!pip install accelerate
!pip install accelerate>=0.21.0
!pip install pdfplumber



/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [36]:
import accelerate
print(accelerate.__version__)


0.30.1


In [37]:
# Imports necessários
import PyPDF2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import ne_chunk
from nltk import pos_tag
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import PyPDF2
import pdfplumber # or import camelot
# Baixe os recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [38]:

def extrair_texto_pdf(caminho_arquivo):
    texto = ""
    with pdfplumber.open(caminho_arquivo) as arquivo: # or with camelot.read_pdf(caminho_arquivo, pages='1-end', flavor='stream') as arquivo:
        for pagina in arquivo.pages:
            texto += pagina.extract_text() # or texto += pagina.df.to_string(header=False, index=False)
    return texto

def processar_texto(texto):
    # Tokenização
    tokens = word_tokenize(texto)

    # Remoção de stopwords
    stop_words = set(stopwords.words('portuguese'))
    tokens_sem_stopwords = [word for word in tokens if word.lower() not in stop_words]

    # Part-of-Speech Tagging
    tagged_tokens = pos_tag(tokens_sem_stopwords)

    # Reconhecimento de Entidades Nomeadas (NER)
    entidades = ne_chunk(tagged_tokens)

    return entidades

# Caminho do arquivo PDF que você deseja ler
caminho_pdf = '/content/clt_e_normas_correlatas_1ed.pdf'

# Extrair texto do PDF
texto_extraido = extrair_texto_pdf(caminho_pdf)

# Processar texto com NLTK (incluindo NER)
entidades_processadas = processar_texto(texto_extraido)

# Exibir entidades processadas
print(entidades_processadas)


(S
  (GSP Consolidação/NNP)
  (ORGANIZATION Leis/NNP Trabalho/NNP)
  –/NNP
  CLT/NNP
  normas/MD
  correlatas/VB
  (PERSON Atualizada/NNP)
  dezembro/VB
  2017Consolidação/CD
  Leis/NNP
  Trabalho/NNP
  –/NNP
  CLT/NNP
  normas/MD
  correlatasSENADO/VB
  (ORGANIZATION FEDERAL/NNP Mesa/NNP)
  Biênio/NNP
  2017/CD
  –/NN
  2018/CD
  (ORGANIZATION Senador/NNP Eunício/NNP Oliveira/NNP)
  PRESIDENTE/NNP
  Senador/NNP
  Cássio/NNP
  Cunha/NNP
  Lima/NNP
  PRIMEIRO-VICE-PRESIDENTE/NNP
  Senador/NNP
  João/NNP
  (PERSON Alberto/NNP Souza/NNP)
  SEGUNDO-VICE-PRESIDENTE/NNP
  Senador/NNP
  José/NNP
  Pimentel/NNP
  PRIMEIRO-SECRETÁRIO/NNP
  Senador/NNP
  (PERSON Gladson/NNP Cameli/NNP)
  SEGUNDO-SECRETÁRIO/NNP
  Senador/NNP
  (PERSON Antonio/NNP Carlos/NNP Valadares/NNP)
  TERCEIRO-SECRETÁRIO/NNP
  Senador/NNP
  Zeze/NNP
  Perrella/NNP
  QUARTO-SECRETÁRIO/NNP
  SUPLENTES/NNP
  SECRETÁRIO/NNP
  Senador/NNP
  (PERSON
    Eduardo/NNP
    Amorim/NNP
    Senador/NNP
    Sérgio/NNP
    Petecão/NNP
   

In [42]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import PyPDF2

# Função para extrair texto do PDF
def extrair_texto_pdf(caminho_arquivo):
    texto = ""
    with open(caminho_arquivo, 'rb') as arquivo:
        leitor_pdf = PyPDF2.PdfFileReader(arquivo)
        num_paginas = leitor_pdf.numPages
        for pagina in range(num_paginas):
            texto += leitor_pdf.getPage(pagina).extractText()
    return texto

# Definir dataset customizado
class TextDataset(Dataset):
    def __init__(self, text, tokenizer, max_length=512):
        self.text = text
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        return self.tokenizer(self.text[idx], max_length=self.max_length, padding="max_length", truncation=True)

# Caminho do arquivo PDF que você deseja usar como base de conhecimento
caminho_pdf = '/content/clt_e_normas_correlatas_1ed.pdf'

# Extrair texto do PDF
texto_extraido = extrair_texto_pdf(caminho_pdf)

# Inicializar tokenizador e modelo
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Adicionar token de padding
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Preparar dados
dataset = TextDataset([texto_extraido], tokenizer)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Parâmetros de treinamento
num_train_epochs = 100
learning_rate = 5e-5

# Otimizador
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Treinamento do modelo
model.train()
for epoch in range(num_train_epochs):
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        inputs = torch.tensor(batch["input_ids"]).to(model.device)
        labels = torch.tensor(batch["input_ids"]).to(model.device)
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(dataloader)}")

# Salvar modelo fine-tuned
output_dir = "./fine_tuned_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


Epoch 1: 100%|██████████| 1/1 [00:16<00:00, 16.30s/it]


Epoch 1 Loss: 4.335720062255859


Epoch 2: 100%|██████████| 1/1 [00:12<00:00, 12.56s/it]


Epoch 2 Loss: 3.7559854984283447


Epoch 3: 100%|██████████| 1/1 [00:13<00:00, 13.08s/it]


...

Epoch 15 Loss: 1.7665181159973145


Epoch 16: 100%|██████████| 1/1 [00:13<00:00, 13.67s/it]


Epoch 16 Loss: 1.6884433031082153


Epoch 17:   0%|          | 0/1 [00:09<?, ?it/s]


KeyboardInterrupt: 

outra versão do código de treinmaento, mas ele não mostrava as épocas e parecia não fazer o processo de treinamento from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import PyPDF2

# Função para extrair texto do PDF
def extrair_texto_pdf(caminho_arquivo):
    texto = ""
    with open(caminho_arquivo, 'rb') as arquivo:
        leitor_pdf = PyPDF2.PdfFileReader(arquivo)
        num_paginas = leitor_pdf.numPages
        for pagina in range(num_paginas):
            texto += leitor_pdf.getPage(pagina).extractText()
    return texto

# Definir dataset customizado
class TextDataset(Dataset):
    def __init__(self, text, tokenizer, max_length=512):
        self.text = text
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        return self.tokenizer(self.text[idx], max_length=self.max_length, padding="max_length", truncation=True)

# Caminho do arquivo PDF que você deseja usar como base de conhecimento
caminho_pdf = '/content/clt_e_normas_correlatas_1ed.pdf'

# Extrair texto do PDF
texto_extraido = extrair_texto_pdf(caminho_pdf)

# Inicializar tokenizador e modelo
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Adicionar token de padding
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Preparar dados
dataset = TextDataset([texto_extraido], tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Parâmetros de treinamento
num_train_epochs = 3
learning_rate = 5e-5

# Otimizador
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Treinamento do modelo
model.train()
for epoch in range(num_train_epochs):
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        inputs = torch.tensor(batch["input_ids"]).to(model.device)
        labels = torch.tensor(batch["input_ids"]).to(model.device)
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(dataloader)}")

# Salvar modelo fine-tuned
output_dir = "./fine_tuned_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


In [40]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Carregar modelo fine-tuned e tokenizador
model_path = "./fine_tuned_model"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Função para gerar resposta do chatbot
def gerar_resposta(pergunta, model, tokenizer, device, max_length=50):
    inputs = tokenizer.encode("Pergunta: " + pergunta, return_tensors="pt").to(device)

    # Set the attention mask and pad token ID
    attention_mask = inputs.ne(tokenizer.pad_token_id)
    inputs = {
        "input_ids": inputs,
        "attention_mask": attention_mask,
        "pad_token_id": tokenizer.pad_token_id,
    }

    resposta_ids = model.generate(**inputs, max_length=max_length, num_return_sequences=1, temperature=0.7)
    resposta = tokenizer.decode(resposta_ids[0], skip_special_tokens=True)
    return resposta

# Loop do chatbot
while True:
    pergunta = input("Você: ")
    if pergunta.lower() == "adeus":
        print("Chat encerrado.")
        break
    resposta = gerar_resposta(pergunta, model, tokenizer, device)
    print("Chatbot:", resposta)


Você: pagamento


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Chatbot: Pergunta: pagamento de 2017
Alcolumbre: 2017
Brasília: 2017
Brasília-Brasília: 2017
Brasília-Brasília-Brasília
Você: tristeza 
Chatbot: Pergunta: tristeza  e normas de   
e normas de normas de normas de normas de normas de normas de normas de normas de normas de normas de normas


KeyboardInterrupt: Interrupted by user